In [7]:
import pandas
import os
import re
import json
from nltk.corpus import wordnet

In [20]:
global matches
matches={}
with open("input.txt",'r') as input_file:
    input_obj=json.load(input_file)
    input_schema = input_obj["schema"]
    input_cat=input_obj["categories"]
    onto=generate_schema_ontology()
    total=0
    for i in input_schema:
        total+=1
    find_tables()

    
    
    
    
#     if "tablename" in input_obj:
#         input_table = input_obj["tablename"]
#         searchTable()
#     else:
#         find_tables()

In [9]:
def get_synonyms(word):
    meanings=set()
    for synset in wordnet.synsets(word,pos=wordnet.NOUN):
        for lemma in synset.lemmas():
            meanings.add(lemma.name())
    for synset in wordnet.synsets(word,pos=wordnet.NOUN):
        for hypernym in synset.hypernyms():
            meanings.add(hypernym.lemma_names()[0])
    meanings.add(word)
    return list(meanings)

In [10]:
def generate_schema_ontology():
    ontology={}
    for col in input_schema:
        ontology[col]=get_synonyms(col)
    return ontology

In [19]:
def find_tables():
    op = open("Output_match.txt",'w')
    f=open("final_schema.txt",'r')
    global matches
    for line in f.readlines():
        json_object=json.loads(line)
        filename=json_object["filename"]
        tablename=json_object["tablename"]
        schema=json_object["schema"]
        category=json_object["categories"]
        ctr=0
        columns=[]
        cats=[]
        for i in input_cat:
            if i in category:
                ctr+=1
                cats.append(i)
        if ctr>1:
            count=0
            for col,dtype in schema.items():
                if (col in input_schema) and (input_schema[col]==dtype):
                    count+=1
                    
                else:
                    for i in onto:
                        if (col in onto[i]) and (input_schema[i]==dtype):
                            count+=1
            if(count>1):
                matches[filename]=(schema,category)
                op.write(filename+"\t\t"+tablename+"\t\t\t\t match score "+str(count)+" of "+str(total)+"\n")        

In [1]:
def generate_list_ontology(list1):
    ontology={}
    for word in list1:
        ontology[word]=get_synonyms(word)
    return ontology

In [13]:
len(matches)

6

In [13]:
def searchAll():
    op = open("Output_match.txt",'w')
    op.write("Format of output is : Table_Name    Column_Name    Data_Type\n")
    f=open("all_schemas_json.txt",'r')
    for line in f.readlines():
        json_object=json.loads(line)
        table=json_object["tablename"]
        schema=json_object["schema"]
        count=0
        for col,dtype in schema.items():
            if (col in input_schema) and (input_schema[col]==dtype):
                count+=1
                op.write(table+'\t\t\t'+col+'\t\t\t'+dtype+'\n')
        global total
        if count != 0:
            op.write(table+" match score "+str(count)+" of "+str(total)+"\n")

In [12]:
def searchTable():
    op = open("Output_match.txt",'w')
    op.write("Format of output is : Table_Name    Column_Name    Data_Type\n")
    f=open("all_schemas_json.txt",'r')
    for line in f.readlines():
        json_object=json.loads(line)
        table=json_object["tablename"]
        schema=json_object["schema"]
        if table in input_table:
            count=0
            for col,dtype in schema.items():
                if (col in input_schema) and (input_schema[col]==dtype):
                    count+=1
                    op.write(table+'\t\t\t'+col+'\t\t\t'+dtype+'\n')
            global total
            op.write(table+" match score "+str(count)+" of "+str(total)+"\n")